# Use codes below when you allot process in GPU server

In [1]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# Import necessary libaries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import oandapyV20

# Read a CSV file and set 'time' column as indices

In [5]:
df = pd.read_csv("../Datasets/20020507_060000_20200207_070000_D.csv",index_col='time',parse_dates=True)
df

,open,high,low,close,amount,complete
time,,,,,,
2002-05-07 06:00:00,127.920,127.920,127.920,127.920,1.0,True
2002-05-08 06:00:00,128.920,128.920,128.920,128.920,1.0,True
2002-05-09 06:00:00,128.380,128.380,128.380,128.380,1.0,True
2002-05-10 06:00:00,127.630,127.630,127.630,127.630,1.0,True
2002-05-11 06:00:00,127.620,127.620,127.620,127.620,1.0,True
...,...,...,...,...,...,...
2020-02-03 07:00:00,108.432,108.799,108.318,108.683,85789.0,True
2020-02-04 07:00:00,108.688,109.546,108.550,109.525,66432.0,True
2020-02-05 07:00:00,109.524,109.848,109.304,109.830,77095.0,True


# Drop 'complete' column which is unnecessary info

In [6]:
df_new = df.drop(columns=['complete'])
df_new

,open,high,low,close,amount
time,,,,,
2002-05-07 06:00:00,127.920,127.920,127.920,127.920,1.0
2002-05-08 06:00:00,128.920,128.920,128.920,128.920,1.0
2002-05-09 06:00:00,128.380,128.380,128.380,128.380,1.0
2002-05-10 06:00:00,127.630,127.630,127.630,127.630,1.0
2002-05-11 06:00:00,127.620,127.620,127.620,127.620,1.0
...,...,...,...,...,...
2020-02-03 07:00:00,108.432,108.799,108.318,108.683,85789.0
2020-02-04 07:00:00,108.688,109.546,108.550,109.525,66432.0
2020-02-05 07:00:00,109.524,109.848,109.304,109.830,77095.0


# Split data to training data and test data

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_new, test_size=0.20, shuffle=False)

In [8]:
train

,open,high,low,close,amount
time,,,,,
2002-05-07 06:00:00,127.920,127.920,127.920,127.920,1.0
2002-05-08 06:00:00,128.920,128.920,128.920,128.920,1.0
2002-05-09 06:00:00,128.380,128.380,128.380,128.380,1.0
2002-05-10 06:00:00,127.630,127.630,127.630,127.630,1.0
2002-05-11 06:00:00,127.620,127.620,127.620,127.620,1.0
...,...,...,...,...,...
2015-12-16 07:00:00,121.691,122.432,121.384,122.208,48842.0
2015-12-17 07:00:00,122.236,122.877,122.198,122.560,29109.0
2015-12-18 07:00:00,122.551,123.566,121.060,121.163,71043.0


In [9]:
test

,open,high,low,close,amount
time,,,,,
2015-12-23 07:00:00,121.058,121.137,120.814,120.918,12937.0
2015-12-24 07:00:00,120.930,120.982,120.250,120.283,16225.0
2015-12-28 07:00:00,120.282,120.628,120.167,120.393,14329.0
2015-12-29 07:00:00,120.404,120.508,120.236,120.460,12195.0
2015-12-30 07:00:00,120.450,120.656,120.352,120.534,10635.0
...,...,...,...,...,...
2020-02-03 07:00:00,108.432,108.799,108.318,108.683,85789.0
2020-02-04 07:00:00,108.688,109.546,108.550,109.525,66432.0
2020-02-05 07:00:00,109.524,109.848,109.304,109.830,77095.0


In [10]:
train.shape, test.shape

((4348, 5), (1087, 5))

# Set window size

In [12]:
window_len = 72

# Preprocess the data

In [13]:
train_lstm_in = []
for i in range(len(train)-window_len):
    temp = train[i:(i+window_len)].copy()
    for col in train:
        temp.loc[:, col] = temp[col] / temp[col].iloc[0] - 1
    train_lstm_in.append(temp)
lstm_train_out = (train['close'][window_len:].values / train['close'][:-window_len].values)-1

test_lstm_in = []
for i in range(len(test) - window_len):
    temp = test[i:(i + window_len)].copy()
    for col in test:
        temp.loc[:, col] = temp[col] / temp[col].iloc[0] - 1
    test_lstm_in.append(temp)
lstm_test_out = (test['close'][window_len:].values / test['close'][:-window_len].values)-1

# Convert the lists into numpy arrays

In [17]:
train_lstm_in = [np.array(train_lstm_input) for train_lstm_input in train_lstm_in]
train_lstm_in = np.array(train_lstm_in)

test_lstm_in = [np.array(test_lstm_input) for test_lstm_input in test_lstm_in]
test_lstm_in = np.array(test_lstm_in)

# (The length of data - Window size, Window size, The number of features)

In [21]:
train_lstm_in.shape

(4276, 72, 5)

In [22]:
test_lstm_in.shape

(1015, 72, 5)

# Import keras libraries

In [24]:
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


# Click the link below to check library info about LSTM
# <a href="https://keras.io/layers/recurrent/">Keras Documentation about LSTM</a>

In [25]:
def build_model(inputs, output_size, neurons, activ_func="linear",
               dropout=0.25, loss="mae", optimizer="adam"):
    model = Sequential()
    
    model.add(LSTM(neurons, return_sequences=True, input_shape = (inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))

    model.add(LSTM(neurons, return_sequences=True))
    model.add(Dropout(dropout))

    model.add(LSTM(neurons, return_sequences=True))
    model.add(Dropout(dropout))

    model.add(LSTM(neurons))
    model.add(Dropout(dropout))

    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
    
#     model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2])))
#     model.add(Dropout(dropout))
#     model.add(Dense(units=output_size))
#     model.add(Activation(activ_func))
    
    model.compile(loss=loss, optimizer=optimizer)
    return model

np.random.seed(202)

yen_model = build_model(train_lstm_in, output_size=1, neurons=50)

yen_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 72, 50)            11200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 72, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 72, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 72, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 72, 50)            20200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 72, 50)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)               

In [ ]:
yen_history = yen_model.fit(train_lstm_in, lstm_train_out, epochs=100, batch_size=32, verbose=1, shuffle=True)

# Visualize the loss function

In [ ]:
fig, ax1 = plt.subplots(1,1)

ax1.plot(yen_history.epoch, yen_history.history['loss'])
ax1.set_title('TrainingError')

if yen_model.loss == 'mae':
    ax1.set_ylabel('Mean Absolute Error (MAE)',fontsize=12)
else:
    ax1.set_ylabel('Model Loss',fontsize=12)
ax1.set_xlabel('# Epochs',fontsize=12)
plt.show()

# Visualize the Training data and Prediction

In [ ]:
from statsmodels.tools.eval_measures import rmse
fig, ax1 = plt.subplots(1,1,figsize=(12,8))
ax1.plot(train.index[window_len:],
    train['close'][window_len:], label='Actual', color='blue', )
ax1.legend()
ax1.plot(train.index[window_len:],
    ((np.transpose(yen_model.predict(train_lstm_in))+1)*train['close'].values[:-window_len])[0],
            label='Predicted', color='red')
ax1.legend()
ax1.set_title('Prediction for Training Data({})'.format(instrument))
ax1.grid(True)

error_train = rmse(train['close'][window_len:], ((np.transpose(yen_model.predict(train_lstm_in))+1)*train['close'].values[:-window_len])[0])
print('STD of train data[close]: {}'.format(train['close'].std()))
print(f'RMSE: {error_train}')
per = round(error_train/train['close'].std()*100, 2)
print('Percentage RMSE/STD: {}%'.format(per))

# Visualize the Test data and Prediction

In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(12,8))
ax1.plot(test.index[window_len:],
    test['close'][window_len:], label='Actual', color='blue')
ax1.legend()
ax1.plot(test.index[window_len:],
    ((np.transpose(yen_model.predict(test_lstm_in))+1)*test['close'].values[:-window_len])[0],
            label='Predicted', color='red')
ax1.legend()
ax1.set_title('Prediction for Test Data({})'.format(instrument))
ax1.grid(True)

error_test = rmse(test['close'][window_len:], ((np.transpose(yen_model.predict(test_lstm_in))+1)*test['close'].values[:-window_len])[0])
print('STD of test data[close]: {}'.format(test['close'].std()))
print(f'RMSE: {error_test}')
per = round(error_test/test['close'].std()*100, 2)
print('Percentage RMSE/STD: {}%'.format(per))

# Use the code below to save the model

In [ ]:
# yen_model.save("LSTM_FX_Model")

# Use the code below to load the saved model file

In [ ]:
# from keras.models import load_model
# model = load_model("LSTM_FX_Model")
# model

In [ ]:
# model.summary()